In [1]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from numpy  import array
from scipy import stats
from gensim.models.callbacks import CallbackAny2Vec

In [2]:
# logging.basicConfig()

# PREPROCESSING

In [3]:
corpus_filename = '../data/extracted_emoji_sequences.txt'

In [4]:
corpus = open(corpus_filename).read().splitlines()


In [5]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

In [6]:
def onlyEmojiSequences(tokens):
    threshold_emojis = [x for x in tokens if len(x) > 1]
    return threshold_emojis

In [7]:
tokenized_corpus = tokenize_corpus(corpus)
emojiSequences = onlyEmojiSequences(tokenized_corpus)

In [8]:
token_count = sum([len(sentence) for sentence in emojiSequences])
print("The corpus contains {0:,} emoji points.".format(token_count))

The corpus contains 610,256 emoji points.


# SETTING VARIABLES

In [9]:
# Dimensionality of the resulting word vectors.
num_features = 300

# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()
# num_workers = 1


# Context window length.
context_size = 4

# Downsample setting for frequent words.
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
# remove later
seed = 1

emoji2vec = w2v.Word2Vec(
    sg=1,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    def __init__(self):
        self.epoch = 0
        self.tempLoss = 0
    def on_epoch_end(self, model):
        print("Epoch #{}  Loss-Value: {}".format(self.epoch, (emoji2vec.get_latest_training_loss() - self.tempLoss)))
        self.epoch += 1
        self.tempLoss = emoji2vec.get_latest_training_loss()
        
epoch_logger = EpochLogger()
emoji2vec.build_vocab(emojiSequences, progress_per=2)

# TRAINING

In [10]:
emoji2vec.train(emojiSequences, total_examples=emoji2vec.corpus_count, epochs = 10, compute_loss=True, callbacks=[epoch_logger])

Epoch #0  Loss-Value: 137791.21875
Epoch #1  Loss-Value: 85361.34375
Epoch #2  Loss-Value: 110157.34375
Epoch #3  Loss-Value: 97510.09375
Epoch #4  Loss-Value: 85497.3125
Epoch #5  Loss-Value: 82586.8125
Epoch #6  Loss-Value: 71961.8125
Epoch #7  Loss-Value: 79175.6875
Epoch #8  Loss-Value: 93126.9375
Epoch #9  Loss-Value: 86470.625
Epoch #10  Loss-Value: 81008.5625
Epoch #11  Loss-Value: 78850.25
Epoch #12  Loss-Value: 92518.25
Epoch #13  Loss-Value: 78041.125
Epoch #14  Loss-Value: 75607.75
Epoch #15  Loss-Value: 64812.875
Epoch #16  Loss-Value: 77415.125
Epoch #17  Loss-Value: 66904.125
Epoch #18  Loss-Value: 71637.875
Epoch #19  Loss-Value: 72455.375
Epoch #20  Loss-Value: 74825.125
Epoch #21  Loss-Value: 79501.0
Epoch #22  Loss-Value: 79164.5
Epoch #23  Loss-Value: 73902.5
Epoch #24  Loss-Value: 75558.375
Epoch #25  Loss-Value: 80786.75
Epoch #26  Loss-Value: 84774.25
Epoch #27  Loss-Value: 63922.5
Epoch #28  Loss-Value: 79881.5
Epoch #29  Loss-Value: 75832.5
Epoch #30  Loss-Value

In [17]:
if not os.path.exists("trained"):
    os.makedirs("trained")
emoji2vec.save(os.path.join("trained", "canIseed.w2v"))

# EVALUATING TRAINED MODEL

In [18]:
emoji2vec = w2v.Word2Vec.load(os.path.join("trained", "canIseed.w2v"))

In [19]:
# extracts the 508 Annotator Results as the Gold-Standard
corpus_filename = '../data/EmoSim508.json'
corpus = open(corpus_filename).read()
annotator_similarity_score_508 = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# glyph_pairs_1016
unicode_pairs_1016 = re.findall('(?<=unicodelong": "\\\)(.*?)(?=")', corpus)    
glyph_pairs_1016 = [codecs.decode(unicode_pairs_1016[x].replace(str('\\\\'),str('\\')).replace('_',''), 'unicode_escape') for x in range(len(unicode_pairs_1016))]

# computation of Cosine Similarity
goldstandard = []
selftrained = []
for x in range(len(annotator_similarity_score_508)):
    cosineSimilarity = None
    
    emoji1 = glyph_pairs_1016.pop(0)
    emoji2 = glyph_pairs_1016.pop(0)
    
    try:
        cosineSimilarity = emoji2vec.wv.similarity(emoji1, emoji2)
    except:
        print('the cosine similarity between ' + emoji1 + ' and ' + emoji2 + ' could not be computed.')
    
    if(cosineSimilarity is not None):
        selftrained.append(cosineSimilarity)
        goldstandard.append(annotator_similarity_score_508.pop(0))

# computation of SPEARRANK CORRELATION COEFFICIENT
spearmanRank = stats.spearmanr(goldstandard, selftrained)

print('Der Spearman Rank Correlation Coefficient is {}'.format(spearmanRank))

the cosine similarity between 🇬🇧 and 🇺🇸 could not be computed.
the cosine similarity between 🏅 and 🇺🇸 could not be computed.
the cosine similarity between 🇺🇸 and ❤ could not be computed.
the cosine similarity between 🇺🇸 and 💥 could not be computed.
the cosine similarity between 🎤 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 📲 could not be computed.
the cosine similarity between 👇 and 🇳🇬 could not be computed.
the cosine similarity between 🎧 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 🎶 could not be computed.
the cosine similarity between 👏 and ↪ could not be computed.
Der Spearman Rank Correlation Coefficient is SpearmanrResult(correlation=0.520770001820602, pvalue=5.776830608808171e-36)


In [20]:
# emoji2vec.wv.get_vector('🙁')
# emoji2vec.wv.similarity('🙁', '🤒')
# emoji2vec.wv.most_similar('💦')